In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from rich.console import Console
from rich.progress import track

In [7]:
LEN = 1807 # Количество компаний, представленных в каталоге Тинькофф Инвестиций

 # Парсинг и структурирование данных в DF

In [8]:
HEADERS = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36" }

In [9]:
def get_HTML(URL, params = None):
    r = requests.get(URL, headers = HEADERS, params = params)
    return r

In [10]:
def curr_parse(URL):
    curr_dict = {}
    html = get_HTML(URL)
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, "html.parser")
        items = soup.find_all("div", class_ = "SecurityColumn__cellPriceSecurities_cswKs")
        curr_dict["$"] = float(items[0].text.replace("\xa0₽","").replace(",","."))
        curr_dict["€"] = float(items[1].text.replace("\xa0₽","").replace(",","."))
        curr_dict["₽"] = 1.0
    return curr_dict
curr_dict = curr_parse("https://www.tinkoff.ru/invest/currencies/")

IndexError: list index out of range

In [11]:
def parse(URL,Len):
    stock_params_dict = {}
    html = get_HTML(URL)
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, "html.parser")
        items = soup.find_all("a", class_ = "Link-module__link__Wv1b Link-module__link_theme_default_gf5wq")
        
        stock_title_list = pd.Series([item.find('div', class_ = "Caption__caption_cyYZT") \
                         for item in items if item.find('div', class_ = "Caption__caption_cyYZT") ]).apply(lambda i: i.text)
        stock_ticker_list = pd.Series([item.find('div', class_ = "Caption__subcaption_xTAKS") \
                         for item in items if item.find('div', class_ = "Caption__subcaption_xTAKS") ]).apply(lambda i: i.text)
        stock_price_list = [price.replace("\xa0","") for price in pd.Series([item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") \
                         for item in items if item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") ]).apply(lambda i: i.text)]
        stock_yield_listok = pd.Series([item.find('span', class_ = "Money-module__money_UwC2N") \
                         for item in items if item.find('span', class_ = "Money-module__money_UwC2N") ]).apply(lambda i: i.text)
        stock_yield_list = [yld.replace("\xa0","") for yld in stock_yield_listok[::2]]
        for i in range(Len):
            stock_params_dict[stock_title_list[i]] = [stock_ticker_list[i],stock_price_list[i], stock_yield_list[i] ]
    else:
        print("Error")
    return  stock_params_dict


In [12]:
console = Console()
with console.status("Parsing...", spinner = "moon"):
    data = parse("https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=" + str(LEN), LEN)

Output()

KeyError: 0

In [18]:
def struct(data):
    all_df = np.transpose(pd.DataFrame(data))
    all_df.columns = ["Ticker","Price","Nom_Yield"]
    all_df["Currency"] = all_df["Nom_Yield"].apply(lambda i: i[-1])
    all_df["Nom_Yield"] = all_df["Nom_Yield"].apply(lambda i: float(i.replace("+","").replace("−","-").replace(",",".")[:-1]))
    all_df["Price"] = all_df["Price"].apply(lambda i: float(i.replace(",",".")[:-1]))
    all_df["Yield_rate"] = (all_df["Price"] / (all_df["Price"] - all_df["Nom_Yield"])).apply(lambda i: round(i, 2) - 1)
#     all_df["Currency_in_RUB"] = all_df["Currency"].apply(lambda i: curr_dict[i])
#     all_df["Price_in_RUB"] = all_df["Price"] * all_df["Currency_in_RUB"]
#     all_df["Nom_Yield_in_RUB"] = all_df["Nom_Yield"] * all_df["Currency_in_RUB"]
    all_df = all_df.where(all_df.notna(), None)
    return all_df


In [19]:
df = struct(data)

NameError: name 'data' is not defined

# Выгрузка DF в Google Sheets

In [ ]:
credentials = ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('test-333300-1860a35c4df3.json', scope)
gc = gspread.authorize(credentials)



In [ ]:
ws = gc.open("test").get_worksheet(0)
ws.update([df.columns.values.tolist()] + df.values.tolist())
ws

# Добавим финансовые показатели с Yohoo Finance и SMART-LAB

In [ ]:
import yfinance as yf

In [ ]:
ticker_list = df.Ticker.tolist()

In [ ]:
params_list = ["Net Income","Gross Profit",'Total Revenue','Research Development' ] 

mslf_example = yf.Ticker("AAPL").financials.transpose()[params_list]
mslf_example

In [ ]:
params_list = ["Net Income","Gross Profit",'Total Revenue','Research Development' ] 
stock_params= mslf[params_list]
stock_params

In [ ]:
%%timeit
bs = yf.Ticker("AAPL").get_balancesheet().transpose()

In [ ]:
data_sber = pd.read_csv("https://smart-lab.ru/q/SBER/f/y/MSFO/download/", ";")

In [ ]:
data_sber_melted = pd.melt(data_sber,  id_vars = ["Unnamed: 0"], value_vars = [str(x) for x in range(2008,2021)].append("LTM"))

In [ ]:
data_sber_melted =  data_sber_melted.rename(columns = {"Unnamed: 0":"Parameter", "variable":"Report_year", "value": "Parameter_value"}).set_index("Parameter").transpose()

In [ ]:
params_list = ["Валюта отчета",'Цена акции ао, руб', "Капитализация, млрд руб","EV, млрд руб",'Чистая прибыль, млрд руб', 'Див.выплата, млрд руб', 'Дивиденд, руб/акцию','Чистый долг, млрд руб','Капитализация, млрд руб','ROE, %',
 'ROA, %','Див доход, ао, %', 'Див доход, ап, %', "EPS, руб",'P/E','P/B', "Free Float, %"] 


In [ ]:
data_sber_melted = data_sber_melted[params_list]

In [6]:
data_sber_melted.to_excel("Sber_info.xlsx")

NameError: name 'data_sber_melted' is not defined

In [7]:
for column in data_sber_melted.columns:
    data_sber_melted[column] = data_sber_melted[column].apply(lambda i: i.replace(".", ","))


NameError: name 'data_sber_melted' is not defined